In [78]:
import numpy as np
import pandas as pd
from PIL import Image

In [79]:
import torch
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize


In [80]:
from tqdm.notebook import tqdm

In [81]:
TRAIN_PATH = './input/data/train/data.csv'

In [82]:
class Custom_Dataset(Dataset):
    def __init__(self, path, target, transforms, train=True):
        self.train = train
        self.path = path
        self.transform = transforms
        
        self.data = pd.read_csv(path, index_col=0)
        self.classes = np.sort(self.data[target].unique())
        
        self.y = self.data[target]
        self.X = []
        for path in self.data['path']:
            im = Image.open(path)
            self.X.append(im)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        X, y = self.X[idx], self.y[idx]
        if self.transform:
          X = self.transform(X)
        return X, torch.tensor(y, dtype=torch.long)

In [83]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [84]:
gender_dataset = Custom_Dataset(path=TRAIN_PATH, target='gender', transforms=transform, train=True)
age_dataset = Custom_Dataset(path=TRAIN_PATH, target='age', transforms=transform, train=True)
mask_dataset = Custom_Dataset(path=TRAIN_PATH, target='mask', transforms=transform, train=True)

In [85]:
gender_dataloader = DataLoader(gender_dataset, batch_size=16, shuffle=True, num_workers=2)
age_dataloader = DataLoader(age_dataset, batch_size=16, shuffle=True, num_workers=2)
mask_dataloader = DataLoader(mask_dataset, batch_size=16, shuffle=True, num_workers=2)

In [86]:
image, label = next(iter(gender_dataloader))
label.float()

<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)


tensor([0., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1.])

In [87]:
from torchvision import models

In [88]:
vgg16 = models.vgg16(pretrained=True)

In [89]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'{device} is using!')

cuda:0 is using!


In [90]:
vgg16.to(device)

lr_rate = 1e-3
epochs = 10

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vgg16.parameters(), lr=lr_rate)

## Gender

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

for epoch in range(epochs):
    running_loss = 0
    running_acc = 0
    for index, (image, label) in enumerate(tqdm(gender_dataloader)):
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        logits = vgg16(image)
        _, preds = torch.max(logits, 1)
        loss = loss_fn(logits, label)
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item() * image.size(0)
        running_acc += torch.sum(preds == label.data)
    
    epoch_loss = running_loss / len(gender_dataloader.dataset)
    epoch_acc = running_acc / len(gender_dataloader.dataset)

    print(f'epoch : {epoch}, loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc}')


batch_size = 16

epoch : 10, loss : 0.669, 평균 Accuracy : 0.6140741109848022

## Age

In [96]:
age_dataloader = DataLoader(age_dataset, batch_size=32, shuffle=True, num_workers=4)

In [97]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

for epoch in range(epochs):
    running_loss = 0
    running_acc = 0
    for index, (image, label) in enumerate(tqdm(age_dataloader)):
        image = image.to(device)
        label = label.to(device)

        optimizer.zero_grad()
        logits = vgg16(image)
        _, preds = torch.max(logits, 1)
        loss = loss_fn(logits, label)
        loss.backward()
        optimizer.step()
    
        running_loss += loss.item() * image.size(0)
        running_acc += torch.sum(preds == label.data)
    
    epoch_loss = running_loss / len(gender_dataloader.dataset)
    epoch_acc = running_acc / len(gender_dataloader.dataset)

    print(f'epoch : {epoch}, loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc}')


<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Py


epoch : 0, loss : 0.953, 평균 Accuracy : 0.47566139698028564


<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Py


epoch : 1, loss : 0.906, 평균 Accuracy : 0.46550264954566956


<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Py


epoch : 2, loss : 0.904, 평균 Accuracy : 0.46851852536201477


<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Py


epoch : 3, loss : 0.904, 평균 Accuracy : 0.4651851952075958


<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return X, torch.tensor(y, dtype=torch.long)
<ipython-input-82-be1ecc8a78e6>:23: DeprecationWarning: an integer is required (got type numpy.float64).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Py

KeyboardInterrupt: 